In [1]:
import re
import numpy as np

In [237]:
def load_steps(fname='day22_cubes.txt'):
    with open(fname) as infile:
        data_raw = [l.strip() for l in infile]

    pattern = r'(\w+) x=(.*)\.\.(.*),y=(.*)\.\.(.*),z=(.*)\.\.(.*)'
    steps = []
    for line in data_raw:
        state, xmin, xmax, ymin, ymax, zmin, zmax = re.match(pattern, line).groups()
        state = 1 if state == 'on' else 0
        steps.append((state, (int(xmin), int(xmax)+1),
                      (int(ymin), int(ymax)+1), (int(zmin), int(zmax)+1)))
    return steps

In [238]:
steps = load_steps()

In [239]:
steps[:25]

[(1, (0, 45), (-37, 13), (-13, 32)),
 (1, (-35, 20), (-9, 38), (-14, 40)),
 (1, (-34, 21), (-25, 26), (-38, 8)),
 (1, (-29, 20), (-43, 10), (-18, 33)),
 (1, (-34, 21), (-3, 48), (-45, 7)),
 (1, (-46, 3), (-25, 26), (-1, 48)),
 (1, (-41, 5), (-15, 30), (-14, 37)),
 (1, (-49, 3), (-16, 36), (-15, 34)),
 (1, (-45, 0), (-9, 46), (-12, 38)),
 (1, (-49, -3), (-44, 8), (-41, 12)),
 (0, (-45, -30), (-9, 4), (-30, -15)),
 (1, (-38, 15), (-39, 9), (-42, 11)),
 (0, (-19, -2), (-21, -11), (5, 24)),
 (1, (-35, 14), (-34, 17), (-25, 30)),
 (0, (-46, -27), (-46, -35), (-42, -30)),
 (1, (-19, 29), (-47, 0), (-35, 13)),
 (0, (19, 32), (-22, -6), (-30, -10)),
 (1, (-13, 37), (-9, 42), (-12, 37)),
 (0, (-31, -19), (-41, -31), (-38, -25)),
 (1, (-31, 15), (-37, 10), (-45, 5)),
 (1, (-73938, -62195), (3443, 9035), (-32612, -17020)),
 (1, (11497, 31461), (60006, 90797), (-22240, 13122)),
 (1, (-84785, -70988), (-24232, 2944), (1355, 33301)),
 (1, (-62304, -41673), (20801, 37686), (-72029, -48577)),
 (1, (27

# Part 1

In [240]:
steps = load_steps()
#steps = load_steps('day22_example.txt')
#steps=[(1, (10, 10), (10, 10), (10, 10))]

In [241]:
region = np.zeros((101, 101, 101))
zero = (50, 50, 50)
step_counters = []

for state, xrange, yrange, zrange in steps:
    xi, xf = xrange[0] + zero[0], xrange[1] + zero[0]
    yi, yf = yrange[0] + zero[1], yrange[1] + zero[1]
    zi, zf = zrange[0] + zero[2], zrange[1] + zero[2]
    if (0<= xi <= region.shape[0] and 0<= xf <= region.shape[0] and
        0<= yi <= region.shape[1] and 0<= yf <= region.shape[1] and
        0<= zi <= region.shape[2] and 0<= zf <= region.shape[2]):
        print(xi, xf, yi, yf, zi, zf)
        region[xi:xf, yi:yf, zi:zf] = state
    step_counters.append(region.sum())

50 95 13 63 37 82
15 70 41 88 36 90
16 71 25 76 12 58
21 70 7 60 32 83
16 71 47 98 5 57
4 53 25 76 49 98
9 55 35 80 36 87
1 53 34 86 35 84
5 50 41 96 38 88
1 47 6 58 9 62
5 20 41 54 20 35
12 65 11 59 8 61
31 48 29 39 55 74
15 64 16 67 25 80
4 23 4 15 8 20
31 79 3 50 15 63
69 82 28 44 20 40
37 87 41 92 38 87
19 31 9 19 12 25
19 65 13 60 5 55


In [242]:
int(region.sum())

596989

# Part 2

In [243]:
arr = np.array([(xlo, xhi, ylo, yhi, zlo, zhi)
                for _, (xlo, xhi), (ylo, yhi), (zlo, zhi) in steps])
print(np.min(arr, axis=0)[::2])
print(np.max(arr, axis=0)[1::2])

[-97641 -98161 -93592]
[97741 94724 98699]


### Not enough memory for previous approach

### We gotta keep track of the abstract cuboids
- Easy to figure out sizes of cuboids
- Tricky to figure out the overlaps
- Then applying all overlaps in correct order

How to do it
- Keep a number of active cubes
- Keep a list of cuboid regions that are on
- Whenever adding a new one that's on, check for overlap with existing on-regions
    - If there's overlap, subtract volume from the counter
- Then check for overlap with tracked off-regions (only part of on-cuboids)
    - If there's overlap, add new cuboid that's on
        - Add its volume to the counter
- When adding a new one that's off, check for overlap with all existing on-regions
    - If there's no overlap, ignore it (unlikely)
    - If there's overlap, create a new cuboid to track it (state = off)
        - Attach those to the mother on-cuboid
        - ~~Order is important~~
    - Check all newly created cuboids for overlaps
        - Subtract (volume of created cuboids - volume of their overlaps) from counter

In [425]:
class Cuboid():
    def __init__(self, xrange, yrange, zrange):
        self.xlo, self.xhi = xrange
        self.ylo, self.yhi = yrange
        self.zlo, self.zhi = zrange
        self.offs = []
    
    def __eq__(self, other):
        eq = (self.xlo == other.xlo and
              self.xhi == other.xhi and
              self.ylo == other.ylo and
              self.yhi == other.yhi and
              self.zlo == other.zlo and
              self.zhi == other.zhi)
        return eq
    
    @property
    def ranges(self):
        return ((self.xlo, self.xhi), (self.ylo, self.yhi),
                (self.zlo, self.zhi))
    
    @property
    def volume(self):
        v = ((self.xhi - self.xlo) *
             (self.yhi - self.ylo) *
             (self.zhi - self.zlo))
        assert v > 0
        return v
    
    @property
    def vertices(self):
        vertices = [(x,y,x)
                    for x in (self.xlo, self.xhi)
                    for y in (self.ylo, self.yhi)
                    for z in (self.zlo, self.zhi)]
        return vertices
    
    def overlap(self, other):
        xlo = max(self.xlo, other.xlo)
        ylo = max(self.ylo, other.ylo)
        zlo = max(self.zlo, other.zlo)
        
        xhi = min(self.xhi, other.xhi)
        yhi = min(self.yhi, other.yhi)
        zhi = min(self.zhi, other.zhi)
        
        if xlo < xhi and ylo < yhi and zlo < zhi:
            return Cuboid((xlo, xhi), (ylo, yhi), (zlo, zhi))
        else:
            return False
    
    def is_inside(self, point):
        x,y,z = point
        if (self.xlo < x < self.xhi and 
            self.ylo < y < self.yhi and 
            self.zlo < z < self.zhi):
            return True
        else:
            return False
        
    def __repr__(self):
        return (f'Cuboid(({self.xlo}, {self.xhi}), '
                f'({self.ylo}, {self.yhi}), '
                f'({self.zlo}, {self.zhi}))')
    
    def __hash__(self):
        return hash(self.ranges)

In [584]:
volumes = []

for i, (state, *ranges) in enumerate(steps):
    new = Cuboid(*ranges)
    for sign, other in volumes.copy():
        overlap = new.overlap(other)
        if overlap:
            # Two "positive" volumes overlapping create a negative volume
            # but a positive volume overlapping with a negative one
            # should add back a positive number
            volumes.append((-sign, overlap))
    if state:
        volumes.append((1, new))

volume = sum([c.volume*sign for sign, c in volumes])
print(volume)

In [586]:
raise RuntimeError

RuntimeError: 

### Original solution - The overlaps are too complicated to keep track of (I thought)
Instead, whenever there's an overlap, let's try to split one of the cuboitds into a few smaller ones. Then remove the overlapping part altogether.

Algorithm
- When adding a new on-cube, look for overlaps with existing on-cubes
- For each overlap
    - Find the cube that has at least one vertex inside the other (these vertices should be shared with the overlap cuboid)
    - Take one such vertex and the coordinates of the outer cube
    - Create 8 new volumes, using one of the vertex coordinates and one of the eight vertices of the outer cube
        - Re-sort low and high coordinates to feed numbers into Cuboid class
    - Remove the original outer volume and cuboid that's identical with the overlap (if exists)
    - See if the inner cube overlaps with any of the new cuboids. If so, repeat the process (recursively?)
- When adding an off-cube, do the same, except don't keep any of the non-overlapping off volumes

In [542]:
def split_both(c1, c2):
    new_ranges = []
    for i in range(3):
        lo1, hi1 = c1.ranges[i]
        lo2, hi2 = c2.ranges[i]
        ll = sorted(set([lo1, lo2, hi1, hi2]))
        new_r = []
        for i in range(len(ll)-1):
            new_r.append([ll[i], ll[i+1]])
        new_ranges.append(new_r)
    new_ranges

    c1_children, c2_children = [], []
    for xrange in new_ranges[0]:
        for yrange in new_ranges[1]:
            for zrange in new_ranges[2]:
                new = Cuboid(xrange, yrange, zrange)
                if new.overlap(c1):
                    c1_children.append(new)
                elif new.overlap(c2):
                    c2_children.append(new)

    return c1_children, c2_children

In [590]:
%%timeit
split_both(c1, c2)

99.9 µs ± 3.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [600]:
#steps = load_steps('day22_overlap_example.txt')
steps = load_steps()[:20]

In [605]:
on_cuboids = []

for i, (state, *ranges) in enumerate(steps):
    print(f'Step {i}, {len(on_cuboids)} tracked cuboids')
    
    master_new = Cuboid(*ranges)
    new_cuboids = [master_new]
    non_overlapping = set()
    
    repeat = True
    while repeat:
        repeat = False
        for other in set(on_cuboids) - non_overlapping:
            for new in new_cuboids:
                overlap = new.overlap(other)
                if overlap:
                    news, others = split_both(new, other)
                    
                    #print(f'Attempting to remove {other}')
                    on_cuboids.remove(other)
                    on_cuboids.extend(others)
                    
                    new_cuboids.remove(new)
                    new_cuboids.extend(news)
                    
                    repeat = True
                    break
            if repeat:
                break
            non_overlapping.add(other)
    if state:
        # After everything is said and done, add all the new
        # on cuboids to the global list
        on_cuboids.extend(new_cuboids)

total_volume = sum([c.volume for c in on_cuboids])
print(total_volume)

Step 0, 0 tracked cuboids
Step 1, 1 tracked cuboids
Step 2, 15 tracked cuboids
Step 3, 63 tracked cuboids
Step 4, 111 tracked cuboids
Step 5, 181 tracked cuboids
Step 6, 262 tracked cuboids
Step 7, 479 tracked cuboids
Step 8, 696 tracked cuboids
Step 9, 835 tracked cuboids
Step 10, 1043 tracked cuboids
Step 11, 1071 tracked cuboids
Step 12, 1580 tracked cuboids
Step 13, 1660 tracked cuboids
Step 14, 2392 tracked cuboids
Step 15, 2421 tracked cuboids
Step 16, 3212 tracked cuboids
Step 17, 3278 tracked cuboids
Step 18, 3793 tracked cuboids
Step 19, 3802 tracked cuboids
596989


Try to speed it up by using a hierarchy of volumes and storing the children below the master volume...

In [602]:
on_cuboids = {}

for i, (state, *ranges) in enumerate(steps):
    print(f'Step {i}, {len(on_cuboids)} tracked cuboids')
    
    master_new = Cuboid(*ranges)
    new_cuboids = [master_new]
    #non_overlapping = set()
    
    repeat = True
    while repeat:
        repeat = False
        for new in new_cuboids:
            #for other in set(on_cuboids) - non_overlapping:
            for master_other in on_cuboids:
                if new.overlap(master_other):
                    for other in on_cuboids[master_other]:
                        overlap = new.overlap(other)
                        if overlap:
                            news, others = split_both(new, other)

                            on_cuboids[master_other].remove(other)
                            on_cuboids[master_other].extend(others)

                            new_cuboids.remove(new)
                            new_cuboids.extend(news)

                            repeat = True
                            break
                    if repeat:
                        break
            if repeat:
                break
            #non_overlapping.add(other)
    if state:
        # After everything is said and done, add all the new
        # on cuboids to the global list
        on_cuboids[master_new] = new_cuboids

total_volume = sum([c.volume for c in on_cuboids])
print(total_volume)

Step 0, 0 tracked cuboids
Step 1, 1 tracked cuboids
Step 2, 2 tracked cuboids
Step 3, 3 tracked cuboids
Step 4, 4 tracked cuboids
Step 5, 5 tracked cuboids
Step 6, 6 tracked cuboids
Step 7, 7 tracked cuboids
Step 8, 8 tracked cuboids
Step 9, 9 tracked cuboids
Step 10, 10 tracked cuboids
Step 11, 10 tracked cuboids
Step 12, 11 tracked cuboids
Step 13, 11 tracked cuboids


KeyboardInterrupt: 

### Old, overcomplicated (and wrong) code

In [257]:
counter = 0
on_cuboids = []
overlaps = []

for i, (state, *ranges) in enumerate(steps[:20]):
    if state:
        print('New on-region')
        new = Cuboid(*ranges)
        print(new)
        new_cuboids = [new]
        counter += new.volume
        print(f'Adding {new.volume} to the counter')
        
        for c_on in overlaps:
            # Check overlap with other, pre-registered overlaps
            overlap = new.overlap(c_on)
            if overlap:
                # Add back to the counter cause we'll remove it in next step
                print('Found overlap with another overlap, '
                      f'adding {overlap.volume} to counter')
                counter += overlap.volume
            
        for c_on in on_cuboids:
            # Check overlap with other on-cuboids
            overlap = new.overlap(c_on)
            if overlap:
                print('Found overlap with another on-cube, '
                      f'subtracting {overlap.volume} from the counter')
                counter -= overlap.volume
                overlaps.append(overlap)
            for c_off in c_on.offs:
                # Check overlap with off-areas of on-cuboids
                overlap2 = new.overlap(c_off)
                if overlap2:
                    print('Found overlap with an off-region, '
                          f'adding {overlap2.volume} back to the counter')
                    # If there's overlap with an off-area,
                    # Create a new independend on-cuboid
                    counter += overlap2.volume
                    new_cuboids.append(overlap2)
                    print('And creating new cuboid', overlap2)
        on_cuboids.extend(new_cuboids)
    else:
        print('New off-region')
        new_off = Cuboid(*ranges)
        print(new_off)
        new_offs = []
        for c_on in on_cuboids:
            overlap = new_off.overlap(c_on)
            if overlap:
                print('Found overlap with an on-cube, creating sub-region')
                c_on.offs.append(overlap)
                new_offs.append(overlap)
                
        print('\nChecking for overlaps with other off-regions')
        found_pairs = set()
        for i, off1 in enumerate(new_offs):
            print()
            counter -= off1.volume
            print(f'Subtracting {off1.volume} from the counter')
            for c_on in on_cuboids:
                print('Looking at off-regions added to', c_on)
                for off2 in c_on.offs:
                    print('Found off-region', off2)
                    if off1 != off2:
                        overlap = off1.overlap(off2)
                        if overlap:
                            if ((off1, off2) not in found_pairs and 
                                (off2, off1) not in found_pairs):
                                print('Found overlap with another off-region, '
                                     f'adding {overlap.volume} back to the counter')
                                counter += overlap.volume
                                found_pairs.add((off2, off1))
                                
    print(f'\nCube counter: {counter}')
    print(f'But should be {int(step_counters[i])}')
    if diff := counter - step_counters[i]:
        print(f'That\'s a difference of {diff}')
    print()
    

New on-region
Cuboid((0, 45), (-37, 13), (-13, 32))
Adding 101250 to the counter

Cube counter: 101250
But should be 101250

New on-region
Cuboid((-35, 20), (-9, 38), (-14, 40))
Adding 139590 to the counter
Found overlap with another on-cube, subtracting 19800 from the counter

Cube counter: 221040
But should be 221040

New on-region
Cuboid((-34, 21), (-25, 26), (-38, 8))
Adding 129030 to the counter
Found overlap with another overlap, adding 9240 to counter
Found overlap with another on-cube, subtracting 16758 from the counter
Found overlap with another on-cube, subtracting 41580 from the counter

Cube counter: 300972
But should be 300972

New on-region
Cuboid((-29, 20), (-43, 10), (-18, 33))
Adding 132447 to the counter
Found overlap with another overlap, adding 17100 to counter
Found overlap with another overlap, adding 14700 to counter
Found overlap with another overlap, adding 20482 to counter
Found overlap with another on-cube, subtracting 42300 from the counter
Found overlap wit

Looking at off-regions added to Cuboid((-19, -2), (-21, -11), (5, 13))
Looking at off-regions added to Cuboid((-19, -2), (-15, -11), (5, 13))
Looking at off-regions added to Cuboid((-19, -2), (-16, -11), (5, 13))
Looking at off-regions added to Cuboid((-19, -3), (-21, -11), (5, 12))
Looking at off-regions added to Cuboid((-19, -2), (-21, -11), (5, 11))

Cube counter: 3258799
But should be 407551
That's a difference of 2851248.0

New on-region
Cuboid((-13, 37), (-9, 42), (-12, 37))
Adding 124950 to the counter
Found overlap with another overlap, adding 19360 to counter
Found overlap with another overlap, adding 9240 to counter
Found overlap with another overlap, adding 23100 to counter
Found overlap with another overlap, adding 16720 to counter
Found overlap with another overlap, adding 28215 to counter
Found overlap with another overlap, adding 12540 to counter
Found overlap with another overlap, adding 6384 to counter
Found overlap with another overlap, adding 25707 to counter
Found o

In [ ]:
7669371029

In [258]:
988224.0

988224.0

In [259]:
counter

4853139